In [4]:
import urllib.parse
import json

In [10]:
input = {
"AUTHOR": "Michel Mayor",
"START_DATE": "2000",
"END_DATE": "2010",
"SUBJECT": "(transitions OR transition) AND (sciences OR scientific OR technology OR technologie)"}

In [23]:
def generate_unige_url(input):
    """
    Génère une URL Swisscovery UNIGE à partir des paramètres fournis.

    :param input: Dictionnaire contenant les paramètres (authors, start_date, end_date, subjects).
    :return: URL construite en chaîne de caractères.
    """
    base_url = "https://unige.swisscovery.slsp.ch/discovery/search"
    authors = input.get("AUTHOR", [])  # Concatène les auteurs avec des virgules si plusieurs.
    subjects = input.get("SUBJECT", [])  # Concatène les sujets avec des virgules si plusieurs.
    start_date = input.get("START_DATE", "")
    end_date = input.get("END_DATE", "")

    # Encode en URL
    
    authors_encoded = urllib.parse.quote(authors)
    subjects_encoded = urllib.parse.quote(subjects)
    start_date_encoded = urllib.parse.quote(start_date)
    end_date_encoded = urllib.parse.quote(end_date)
    
    url = (
        f"{base_url}?"
        f"query=creator,contains,{authors_encoded},AND&"
        f"query=any,contains,{subjects_encoded},AND&"
        f"pfilter=dr_s,exact,{start_date_encoded}0101,AND&"
        f"pfilter=dr_e,exact,{end_date_encoded}1231,AND&"
        f"tab=41SLSP_UGE_MyInst_CI&"
        f"search_scope=MyInst_and_CI&"
        f"vid=41SLSP_UGE:VU1&"
        f"mode=advanced"
    )
    
    return url


In [24]:
result_url = generate_unige_url(input)
print(result_url)

https://unige.swisscovery.slsp.ch/discovery/search?query=creator,contains,Michel%20Mayor,AND&query=any,contains,%28transitions%20OR%20transition%29%20AND%20%28sciences%20OR%20scientific%20OR%20technology%20OR%20technologie%29,AND&pfilter=dr_s,exact,20000101,AND&pfilter=dr_e,exact,20101231,AND&tab=41SLSP_UGE_MyInst_CI&search_scope=MyInst_and_CI&vid=41SLSP_UGE:VU1&mode=advanced


In [ ]:
# To improve:
# Take a response that includes not JSON text (ex: explanation before and after the JSON)
# Manage a response with only the start date (i.e. set automatically end_date = stard_date)
# Avoid that the AI put a subject date as a date filter (ex: document about XVIIIe s.)
# Analyze the subjets, find synonyms and launch OR requests